In [1]:
import torch
import matplotlib.pyplot as plt

# Reproducibility
torch.manual_seed(0)

# ==========================
# User-settable parameters
# ==========================

# Period / vector length
p = 256  # e.g. 256 samples per period

device = "cpu"  # or "cuda" if you want & have it

# ---- Amplitude distribution for A_i ----
# Option 1 (recommended): zero-mean Gaussian
amp_dist = "normal"   # "normal" or "uniform"

# For normal
amp_std = 1.0         # standard deviation of A_i

# For uniform
amp_max = 1.0         # A_i ~ U[-amp_max, amp_max]

# ---- FIR filter parameters ----
fir_len = 21          # number of taps in FIR (finite impulse response)
fir_dist = "normal"   # distribution for FIR taps: "normal" or "uniform"
normalize_fir = True  # normalize FIR taps (L1) to keep overall gain reasonable

: 

: 